# Misc

This one was hard to classify

## Precompiles

Precompiles in Ethereum are programs that the EVM can execute directly. They are currently 9 of them.

Implementing these functions in EVM bytecode would be very gas inefficient. So it was decided to include them in the EVM itself.

![title](../../static/precompiles.png)

[source](https://www.evm.codes/precompiled)

## Opcode vs Precompile

The `sha3` could have easily been a precompile. Weather an operation should be a opcode or a precomiple is a matter of debate in the Ethereum community.

#### Hash Function

A hash function is one of the most important cryptographic primitives. It has the following characteristics:

- fixed sized: every input (also called message) creates a hash value of fixed size.
- deterministic: the same input will produce the same output every time.
- one-way: its practically infeasible to invert.
- chaotic: if only one bit changes the whole hash changes in a toatlly chaotic and random way.

Note that we are using the Python built-in `hash` function, so we do not have to import any external library. 

The EVM uses the Keccak-256 function. 

In [ ]:
use sha3::{Digest, Sha3_256};

fn sha3(evm: &mut Evm) {
    let size = evm.stack.pop();
    let offset = evm.stack.pop();
    let value = evm.memory.access(offset, size);
    
    let mut hasher = Sha3_256::new();
    hasher.update(value.to_string().as_bytes());
    let hash_result = hasher.finalize();
    
    evm.stack.push(format!("{:x}", hash_result));
    
    evm.pc += 1;
    
    // calculate gas
    let minimum_word_size = (size + 31) / 32;
    let dynamic_gas = 6 * minimum_word_size; // TODO: + memory_expansion_cost
    evm.gas_dec(30 + dynamic_gas);
}